In [ ]:
# Data availability
##' 

In [2]:
# Loading Packages
import numpy as np
import scanpy as sc
import decoupler as dc
import pandas as pd
import subprocess

/Users/ogawaakane/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# intensity of each phospho-site
intensity_per_psite = pd.read_table("../data/raw_data.txt")
intensity_per_psite

,idx,111,114,124,126,128,132,136,138,142,...,1022,1026,1028,1032,1042,1044,1046,448,536,698
0,ENSG00000003056.8|ENSP00000000412.3|S267|DDQLG...,23.950379,23.592380,23.839427,23.734960,23.703682,23.684060,23.843815,23.568329,23.984569,...,23.285770,23.370740,23.630446,23.258484,23.302306,23.294031,23.647228,23.235143,24.128864,23.558849
1,ENSG00000048028.11|ENSP00000003302.4|S1053|PPT...,25.304118,25.493882,25.552121,24.889130,25.472239,25.115037,25.069740,25.013908,25.184324,...,24.942877,25.251571,25.162289,25.016707,24.888852,24.466110,24.396240,25.071729,25.027466,25.465617
2,ENSG00000004776.13|ENSP00000004982.3|S16|PSWLR...,29.388436,29.097768,29.881585,29.384304,29.000992,27.743867,29.221645,29.219980,28.720471,...,29.961800,29.658444,29.679477,29.207971,29.339685,29.621490,29.320550,29.936182,30.969038,29.281749
3,ENSG00000004975.12|ENSP00000005340.4|S170|RPRR...,22.420002,22.437873,22.690769,22.451642,22.301636,21.741052,22.228490,21.797540,21.567148,...,21.855150,21.651538,21.990740,21.741293,20.981468,21.919813,21.973437,21.861671,21.920173,21.661722
4,ENSG00000006025.12|ENSP00000007414.3|S272|RVGR...,23.497811,23.619235,23.908199,23.923280,23.086429,23.035650,23.598035,23.597075,22.731151,...,23.259723,23.554293,23.563693,23.240707,24.161588,23.904668,23.820420,23.922922,24.561779,24.073079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57914,ENSG00000115109.14|ENSP00000263713.5|S532|NIDV...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57915,ENSG00000166483.11|ENSP00000402084.2|S270|CGED...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57916,ENSG00000089685.15|ENSP00000324180.4|T34|FLEGC...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57917,ENSG00000183856.11|ENSP00000354451.2|S539|NEAL...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# data shaping
# 同じ時点のreplicateを平均化
expr = (
    intensity_per_psite
    .groupby(["psite", "time"], as_index=False)["intensity"]
    .mean()
    .pivot(index="psite", columns="time", values="intensity")
)
gene_names = intensity_per_psite["psite"].unique()
# calculate z score
expr_z = expr.sub(expr.mean(axis=1), axis=0).div(expr.std(axis=1), axis=0) # (値 - 行平均) / 行標準偏差
expr_z = expr_z.dropna(how="any") # NaNを含む行を削除
expr_z

time,0,5,15,30,60,120,240
psite,,,,,,,
AAGAB_S310,-0.561163,0.480397,0.118534,0.935804,-1.556747,-0.701143,1.284319
AAGAB_S311,0.981355,0.409299,0.552500,0.569132,-1.438372,-1.429899,0.355985
AAK1_S20,0.412705,1.072255,0.780696,-1.185925,-1.376875,-0.453137,0.750281
AAK1_S21,0.937020,1.363256,0.608778,-0.167256,-1.300442,-0.980523,-0.460833
AAK1_S624,-0.117705,1.115899,1.176471,-0.730903,-1.562022,-0.374291,0.492550


In [ ]:
# obs
obs = pd.DataFrame(index=expr_z.columns)  
obs.index.name = "sample"  
obs["time"] = obs.index.astype(int)
# expr_z は psite を行、time を列に持つデータフレーム
X = expr_z.T.values  # AnnData の構造に合わせるために転置
# var
var = pd.DataFrame(index=expr_z.index)  # index: psite
var.index.name = "psite"
# AnnData 
adata = sc.AnnData(X=X, obs=obs, var=var)

/Users/ogawaakane/Library/Python/3.9/lib/python/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [8]:
# Import net
subprocess.run(["Rscript", "get_omnipath_ptm.R"], check=True)
omnipath_ptm = pd.read_csv('../data/omnipath_ptm.csv')
omnipath_ptm


次のパッケージを付け加えます: ‘dplyr’

以下のオブジェクトは ‘package:stats’ からマスクされています:

    filter, lag

以下のオブジェクトは ‘package:base’ からマスクされています:

    intersect, setdiff, setequal, union

[2025-09-08 14:20:42] [SUCCESS] [OmnipathR] Loaded 391929 annotation records from cache.
[2025-09-08 14:20:43] [SUCCESS] [OmnipathR] Loaded 43269 enzyme-substrate relationships from cache.
[2025-09-08 14:20:44] [SUCCESS] [OmnipathR] Loaded 81529 interactions from cache.


,target,source,weight,likelihood
0,ITGB2_S745,PRKCH,1,1
1,ITGB2_T758,PRKCH,1,1
2,NPM1_S227,PRKCH,1,1
3,NPM1_T234,PRKCH,1,1
4,GSTP1_S43,PRKCH,1,1
...,...,...,...,...
30811,MAPK9_Y185,MAP3K4,1,1
30812,MAPK9_T183,MAP3K4,1,1
30813,MAPK10_Y223,MAP3K4,1,1
30814,MAPK10_T221,MAP3K4,1,1


In [ ]:
# ===== run ULM =====
dc.run_ulm(
    mat=adata,
    net=omnipath_ptm,
    use_raw=False,     # X を使う
    verbose=True 
)

Running ulm on mat with 7 samples and 25789 targets for 182 sources.


In [ ]:
# ===== get activity scores =====
score = dc.get_acts(adata, obsm_key='ulm_estimate')
score_df = score.to_df()

In [ ]:
# save kinase activities
decoupleR_kin = score_df.T
decoupleR_kin.to_csv('../results/tables/decoupleR_kin.csv', index=True)
decoupleR_kin

sample,0,5,15,30,60,120,240
ABL1,-0.426737,-0.047711,-0.497784,-0.077026,0.542385,-0.323489,0.823894
AKT1,2.470201,0.967131,-2.141115,2.082965,1.142345,-1.937539,-3.164726
AKT2,3.303248,1.604716,-0.886623,0.453115,-0.323548,-1.933987,-3.264504
AKT3,0.848391,1.252373,-1.237064,0.038970,0.407253,-1.127549,-0.649992
ARAF,0.335499,-0.123778,-0.343865,-0.482889,0.595567,-0.777102,0.501792
...,...,...,...,...,...,...,...
VRK1,-2.888578,-1.995531,0.116989,0.370220,1.805935,2.358894,1.535608
VRK2,-2.629191,-1.137611,1.233398,0.116312,0.747887,1.730181,1.181876
WEE1,1.359460,0.731731,0.885033,-0.864500,-0.863791,-0.152395,-1.476892
WNK1,0.319455,0.775506,1.398410,-0.026572,0.774371,-1.078007,-1.795170


In [ ]:
# draw lineplots 
subprocess.run(["Rscript", "kin_lineplot.R"], check=True)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
警告メッセージ:
1: パッケージ ‘ggplot2’ はバージョン 4.4.1 の R の下で造られました 
2: パッケージ ‘purrr’ はバージョン 4.4.1 の R の下で造られました 
3: パッケージ ‘lubridate’ はバージョン 4.4.1 の R の下で造られました 


CompletedProcess(args=['Rscript', '/Users/ogawaakane/Yugi_lab_project/phosphoproteome/scripts/R/kin_lineplot.R', '/Users/ogawaakane/Yugi_lab_project/phosphoproteome'], returncode=0)